In [21]:
import requests
import json
API_KEY = {'X-API-Key': 'KZJ94OPT'}
session = requests.Session()
session.headers.update(API_KEY)

In [11]:
class ApiException(Exception):
    pass

class HelperFunctions():
    
    def __init__(self):
        pass

    def ticker_bid_ask(self, session, ticker):
        payload = {'ticker': ticker}
        resp = session.get('http://localhost:9999/v1/securities/book', params=payload)  
        if resp.ok:
            book = resp.json()
            return book['bids'][0]['price'], book['asks'][0]['price']
        raise ApiException('Error getting bid / ask: The API key provided in this Python code must match that in the RIT client')

In [9]:
array = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
sum = 0
sma_period = 12

for idx, i in reversed(list(enumerate(array))):
    if idx > len(array) - 13:
        sum += i

mov_avg = sum/sma_period
print(mov_avg)

# use this to determine whether to have more quantity / orders on buy side or sell side
# if price > sma -> buy more // vice versa

13.5


In [35]:
resp = session.get('http://localhost:9999/v1/case')
ticker_history = session.get('http://localhost:9999/v1/securities/history', params={'ticker': 'ALGO'})

price = ticker_history.json()[0]['close']
print(price)

array = []
array.append(price)
sum = 0
sma_period = 12

for idx, i in reversed(list(enumerate(array))):
    if idx > len(array) - sma_period + 1:
        sum += i

mov_avg_num = sum/sma_period
print(mov_avg_num)

20.11
1.6758333333333333


In [61]:
def get_order_book_stats(session, ticker, limit):

    orders = session.get('http://localhost:9999/v1/securities/book', params={'ticker': ticker, 'limit': limit})

    bids = orders.json()['bids']
    asks = orders.json()['asks']

    bid_cumulative_volume = 0
    bid_number_of_orders = 0

    ask_cumulative_volume = 0
    ask_number_of_orders = 0

    for i in bids:
        if i['trader_id'] == 'ANON':

            bid_cumulative_volume += i['quantity']
            bid_number_of_orders += 1

        # else:
        #     del bids[i]

    for i in asks:
        if i['trader_id'] == 'ANON':

            ask_cumulative_volume += i['quantity']
            ask_number_of_orders += 1

        # else:
        #     del asks[i]

    dict = {'Cumulative Vol Bid': bid_cumulative_volume,
            'Bid Num of Orders': bid_number_of_orders,
            'Cumulative Vol Ask': ask_cumulative_volume,
            'Ask Num of Orders': ask_number_of_orders,}

    return dict

    # for i in orders.json()['bid']:
    #     if orders.json()['bid']['trader-id'] != 'ANON':

print(get_order_book_stats(session, 'ALGO', 100))

{'Cumulative Vol Bid': 0, 'Bid Num of Orders': 0, 'Cumulative Vol Ask': 0, 'Ask Num of Orders': 0}


In [65]:
print(get_order_book_stats(session, 'ALGO', 100))

x = get_order_book_stats(session, 'ALGO', 100)

def calc_spread_cushion(order_books_stats):
    bid_vol = order_books_stats['Cumulative Vol Bid']
    ask_vol = order_books_stats['Cumulative Vol Ask']

    reduction_factor = 20
    price_cushion_factor = (bid_vol - ask_vol) / ask_vol / reduction_factor

    return price_cushion_factor

print(calc_spread_cushion(x))

{'Cumulative Vol Bid': 3219700.0, 'Bid Num of Orders': 64, 'Cumulative Vol Ask': 2952300.0, 'Ask Num of Orders': 58}
0.004528672560376656
